In [3]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [48]:
summoner_name = "야식은치킨이지"

match = pd.read_json("data/data_summoner_match_data_{name}.json".format(name=summoner_name))
timeline = pd.read_json("data/data_summoner_match_timeline_data_{name}.json".format(name=summoner_name))

# 경기 관련 데이터

In [49]:
match_length = len(match["info"])

In [50]:
info = match["info"]

participants = []
teams = []
for i in range(match_length):
    participants.append(info[i]["participants"])
    teams.append(info[i]["teams"])
    del(info[i]["participants"])
    del(info[i]["teams"])

In [61]:
df_info = pd.json_normalize(info)
df_info_1 = df_info[["gameDuration", "gameMode", "gameType", "gameVersion", "mapId"]]
display(df_info_1.head(10))

,gameDuration,gameMode,gameType,gameVersion,mapId
0,950957,URF,MATCHED_GAME,11.19.398.9466,11
1,904515,URF,MATCHED_GAME,11.19.398.9466,11
2,1121760,URF,MATCHED_GAME,11.19.398.9466,11
3,1370877,URF,MATCHED_GAME,11.19.398.9466,11
4,1287954,URF,MATCHED_GAME,11.19.398.9466,11
5,751931,URF,MATCHED_GAME,11.19.398.9466,11
6,1335078,URF,MATCHED_GAME,11.19.398.9466,11
7,1150752,URF,MATCHED_GAME,11.19.398.9466,11
8,961086,ARAM,MATCHED_GAME,11.15.389.2308,12
9,1943798,CLASSIC,MATCHED_GAME,11.15.389.2308,11


In [64]:
df_participants = pd.json_normalize(participants)
display(df_participants.head(1))

,0,1,2,3,4,5,6,7,8,9
0,"{'assists': 7, 'baronKills': 0, 'bountyLevel':...","{'assists': 13, 'baronKills': 0, 'bountyLevel'...","{'assists': 7, 'baronKills': 0, 'bountyLevel':...","{'assists': 10, 'baronKills': 0, 'bountyLevel'...","{'assists': 8, 'baronKills': 0, 'bountyLevel':...","{'assists': 1, 'baronKills': 0, 'bountyLevel':...","{'assists': 8, 'baronKills': 0, 'bountyLevel':...","{'assists': 2, 'baronKills': 0, 'bountyLevel':...","{'assists': 6, 'baronKills': 0, 'bountyLevel':...","{'assists': 5, 'baronKills': 0, 'bountyLevel':..."


In [67]:
df_teams = pd.json_normalize(teams)
df_teams.columns = ["100", "200"]   # teamId
display(df_teams.head(5))

,100,200
0,"{'bans': [{'championId': 105, 'pickTurn': 1}, ...","{'bans': [{'championId': 7, 'pickTurn': 6}, {'..."
1,"{'bans': [{'championId': 711, 'pickTurn': 1}, ...","{'bans': [{'championId': 7, 'pickTurn': 6}, {'..."
2,"{'bans': [{'championId': 238, 'pickTurn': 1}, ...","{'bans': [{'championId': 81, 'pickTurn': 6}, {..."
3,"{'bans': [{'championId': 238, 'pickTurn': 1}, ...","{'bans': [{'championId': 28, 'pickTurn': 6}, {..."
4,"{'bans': [{'championId': 7, 'pickTurn': 1}, {'...","{'bans': [{'championId': 126, 'pickTurn': 6}, ..."


In [76]:
df_teams["100"][0]["win"]

True

In [69]:
df_info_1.to_csv("data/{}/info.csv".format(summoner_name))
df_participants.to_csv("data/{}/participants.csv".format(summoner_name))
df_teams.to_csv("data/{}/teams.csv".format(summoner_name))

# 인베이드 관련 데이터

In [85]:
match_invade = 0

## 이벤트 분석

In [86]:
is_invade = [False]*match_length
invade_timestamp = [0]*match_length           
invade_kill_blue = [0]*match_length           # invade시 발생한 블루팀의 킬 수
invade_kill_red = [0]*match_length            # invade시 발생한 레드팀의 킬 수
invade_win_team = [0]*match_length            # invade가 발생한 게임에서 이긴 팀의 teamId
total_gold_blue_3_min = [0]*match_length
total_gold_red_3_min = [0]*match_length
gold_gap = [0]*match_length

for match_index in range(match_length):
    match_time_length = len(timeline.iloc[match_index].info['frames'])
    
    events = [timeline.iloc[match_index].info["frames"][t]["events"] for t in range(match_time_length)]
    participants_info = [timeline.iloc[match_index].info["frames"][t]["participantFrames"] for t in range(match_time_length)]

    # 1~2분대의 kill event
    for t in range(1,3):  # 1 ~ 2 분 사이
        for e in events[t]:    
            if e["type"] == "CHAMPION_KILL":
                if is_invade[match_index] == False: 
                    is_invade[match_index] = True
                    match_invade += 1
                if invade_timestamp[match_index] == 0: invade_timestamp[match_index] = e['timestamp']
                if e['killerId'] <= 5:
                    invade_kill_blue[match_index] += 1
                elif e['killerId'] > 5:
                    invade_kill_red[match_index] += 1

    if is_invade[match_index]:
        for i in range(1,6):
            total_gold_blue_3_min[match_index] += participants_info[3][str(i)]['totalGold']
        for i in range(6,11):
            total_gold_red_3_min[match_index] += participants_info[3][str(i)]['totalGold']
        gold_gap[match_index] = total_gold_blue_3_min[match_index] - total_gold_red_3_min[match_index]
        
        # use df_teams
        invade_win_team[match_index] = 100 if df_teams["100"][match_index]["win"] else 200
        
        print("--------------------------------------------------")
        print("invade detected: %d" % match_index)
        print("timestamp(minutes): %.2f" % (invade_timestamp[match_index]/60000))
        print("< kills >\n  blue : %d , red : %d"% (invade_kill_blue[match_index], invade_kill_red[match_index]))
        print("< total_gold_3_min >\n  blue: %d , red: %d"% (total_gold_blue_3_min[match_index], total_gold_red_3_min[match_index]))
        print("< gold_gap >\n  %d"% gold_gap[match_index])
        print("< win >\n  %s"% ("blue" if invade_win_team[match_index] == 100 else "red"))

--------------------------------------------------
invade detected: 0
timestamp(minutes): 1.05
< kills >
  blue : 2 , red : 2
< total_gold_3_min >
  blue: 7461 , red: 6960
< gold_gap >
  501
< win >
  blue
--------------------------------------------------
invade detected: 1
timestamp(minutes): 1.60
< kills >
  blue : 0 , red : 1
< total_gold_3_min >
  blue: 7579 , red: 6743
< gold_gap >
  836
< win >
  blue
--------------------------------------------------
invade detected: 2
timestamp(minutes): 0.43
< kills >
  blue : 3 , red : 7
< total_gold_3_min >
  blue: 8866 , red: 9835
< gold_gap >
  -969
< win >
  blue
--------------------------------------------------
invade detected: 3
timestamp(minutes): 0.74
< kills >
  blue : 0 , red : 1
< total_gold_3_min >
  blue: 7998 , red: 7759
< gold_gap >
  239
< win >
  blue
--------------------------------------------------
invade detected: 4
timestamp(minutes): 1.65
< kills >
  blue : 0 , red : 1
< total_gold_3_min >
  blue: 7114 , red: 7840
< g

In [87]:
df_invade = pd.DataFrame([
    is_invade,
    invade_timestamp,
    invade_kill_blue,
    invade_kill_red,
    invade_win_team,
    total_gold_blue_3_min,
    total_gold_red_3_min,
    gold_gap,
    df_info_1["gameMode"]]).transpose()

df_invade.columns = [
    "is_invade",
    "invade_timestamp",
    "invade_kill_blue",
    "invade_kill_red",
    "invade_win_team",
    "total_gold_blue_3_min",
    "total_gold_red_3_min",
    "gold_gap",
    "gameMode"]

display(df_invade[(df_invade.is_invade==True) & (df_invade.invade_kill_blue > df_invade.invade_kill_red)])

# 인베가 일어났고, 블루가 레드보다 킬을 많이 딴 경기에서 블루가 이길 확률 
print("%.2f%%"%(
    len(df_invade[
        (df_invade.is_invade==True) 
        & (df_invade.invade_kill_blue > df_invade.invade_kill_red) 
        & (df_invade.invade_win_team == 100)]) 
    / len(df_invade[
        (df_invade.is_invade==True) 
        & (df_invade.invade_kill_blue > df_invade.invade_kill_red)])
    * 100)
)


,is_invade,invade_timestamp,invade_kill_blue,invade_kill_red,invade_win_team,total_gold_blue_3_min,total_gold_red_3_min,gold_gap,gameMode
8,True,44425,4,1,100,13290,11933,1357,ARAM
18,True,85817,2,1,100,12770,11884,886,ARAM
19,True,109439,2,1,100,12470,11800,670,ARAM
20,True,73810,4,1,100,13207,11693,1514,ARAM
25,True,67085,1,0,200,9007,7358,1649,ULTBOOK
33,True,85660,1,0,200,8381,7817,564,ULTBOOK
38,True,119040,1,0,200,8404,7791,613,ULTBOOK
39,True,117294,1,0,100,8681,8270,411,ULTBOOK
40,True,58295,2,0,100,10104,6965,3139,ULTBOOK
50,True,80851,2,1,100,13726,12922,804,ARAM


57.69%


In [88]:
match_invade

73

In [84]:
df_invade

,is_invade,invade_timestamp,invade_kill_blue,invade_kill_red,invade_win_team,total_gold_blue_3_min,total_gold_red_3_min,gold_gap,gameMode
0,True,63063,2,2,100,7461,6960,501,URF
1,True,96234,0,1,100,7579,6743,836,URF
2,True,25730,3,7,100,8866,9835,-969,URF
3,True,44363,0,1,100,7998,7759,239,URF
4,True,98734,0,1,200,7114,7840,-726,URF
...,...,...,...,...,...,...,...,...,...
175,False,0,0,0,0,0,0,0,CLASSIC
176,False,0,0,0,0,0,0,0,CLASSIC
177,False,0,0,0,0,0,0,0,CLASSIC
178,False,0,0,0,0,0,0,0,CLASSIC
